### Import raw data

In [1]:
import pandas as pd

data = pd.read_csv('/Users/Hsuweic/Desktop/AI4health/dataset/raw data.csv')
data = data[['tweet','class']]
print(len(data))
print(data.head(10))

24783
                                               tweet  class
0  !!! RT @mayasolovely: As a woman you shouldn't...      2
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...      1
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...      1
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...      1
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...      1
5  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...      1
6  !!!!!!"@__BrighterDays: I can not just sit up ...      1
7  !!!!&#8220;@selfiequeenbri: cause I'm tired of...      1
8  " &amp; you might not get ya bitch back &amp; ...      1
9  " @rhythmixx_ :hobbies include: fighting Maria...      1


In [2]:
hate_speech = data[data['class'] == 0]
hate_speech['class'] = 1
hate_speech.rename(columns={'class':'label'}, inplace=True)
print("Hate speech: " + str(len(hate_speech)))
# print(hate_speech.head(10))

non_hate_speech = data[data['class'] == 2]
non_hate_speech['class'] = 0
non_hate_speech.rename(columns={'class':'label'}, inplace=True)
print("Non hate speech: "+ str(len(non_hate_speech)))
# print(non_hate_speech.head(10))

offensive_speech = data[data['class'] == 1]
print("Offensive speech: "+ str(len(offensive_speech)))

Hate speech: 1430
Non hate speech: 4163
Offensive speech: 19190


/var/folders/j5/b206ytkd1f52lmgd91ffpq1r0000gn/T/ipykernel_59543/726737079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_speech['class'] = 1
/var/folders/j5/b206ytkd1f52lmgd91ffpq1r0000gn/T/ipykernel_59543/726737079.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_speech.rename(columns={'class':'label'}, inplace=True)
/var/folders/j5/b206ytkd1f52lmgd91ffpq1r0000gn/T/ipykernel_59543/726737079.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



### Generate category labels from OpenAI API call

In [3]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("openai_api_key")

In [5]:
category_values = []
idx = 1

for i in hate_speech['tweet']:
    prompt = "What is the category of this tweet '{text}'? Give me one answer from these options: [behavior,class,ethnicity,disability,gender,physical appearance,race,religion,sexual orientation]. Remember you should limit the answer to the above options.".format(text=i)
    # model = "text-davinci-003"
    model = "davinci-002"

    # Generate a response
    response = openai.completions.create(
        model=model,
        prompt=prompt,
        max_tokens=50  # Adjust the max_tokens as needed
    )
    # Extract the generated text from the response
    generated_text = response.choices[0].text
    generated_text = generated_text.replace('\n', '').replace('\n', '')
    # print(generated_text)
    category_values.append(generated_text)
    print(idx)
    idx = idx+1

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [44]:
print(len(category_values))
hate_speech['categories'] = category_values
print(hate_speech.head(10))

non_hate_speech['categories'] = ""
print(non_hate_speech.head(10))

10
                                               tweet  label
0  @SelfCuttingGirl @AnthonyCumia I thought he wo...      0
1  Chaber Books: BRAGADAYJAH 24 http://t.co/bTMLY...      0
2  Polite, yet militant. #birds #notfeeding #sten...      0
3  RT @SportsCenter: Don't look now, but the Brow...      0
4  RT @TeamGladney: Did the Yankees make the play...      0
5  "@TEE_JONEZ: @16stanleys @KingCuh tuku ho idk ...      0
6  @Hunglikerobby_ that was so gay. And I was tan...      1
7  @ActaBunniFooFoo I like my gross yellow heat l...      0
8  RT @ryancruz_: Travis always on my TL being gh...      0
9  Pennsylvania is white trash and is so are all ...      1


In [ ]:
combined_df = pd.concat([hate_speech, non_hate_speech], ignore_index=True)
# combined_df.rename(columns={'class':'label'}, inplace=True)
# print(combined_df.columns)
dataset = combined_df.sample(frac=1).reset_index(drop=True) # shuffle dataset
 
print("Total dataset: "+ str(len(dataset)))
print(dataset.head(10))
# dataset.to_csv("dataset/dataset_categories.csv")

### Check duplicate category labels

In [55]:
final_dataset = pd.read_csv("/Users/Hsuweic/Desktop/AI4healthcare/dataset/dataset_categories.csv")
lst = []
for category in final_dataset["categories"]:
    if category not in lst:
        lst.append(category)

print(lst)
print(len(lst))

[nan, 'Race', 'Sexual Orientation', 'Gender', 'Religion', 'Disability', 'Physical Appearance', 'Class', 'Ethnicity', 'Behavior']
10
